In [ ]:
from molsim import MolecularDynamics
import numpy as np
import matplotlib.pyplot as plt
import time

Currently, the code for the MD package is still quite suboptimal! We can easily add some implementations that will speed up the runs. When optimizing code it is best to optimize the parts that are executed most often. In our case, the energy and force calculation is carried out $N^2$ times, meaning it is quite performance heavy. There are a few things that cost performance and by implementing them we can reach about 10x more efficiency.

You can find the force computation in:
`src/molecularDynamics/md.cpp`
in function:
`MolecularDynamics::calculateForce`

##### Lower triangular force computation
The double loop i, j over the numbers of particles both loop over N, while we only need to calculate the lower triangular part of the matrix. Another reason why this is inefficient is because we need an if statement checking i!=j in the inner loop. If statements make code unpredictable for the computer, leading to reduced optimization. 

We can change the loop from i (0, ... N) and j (0, ... N) by only considering pairs where j is larger than i. The easiest option is replacing (i != j) with (i < j), but this does not remove the if statement. Another way would be to loop i from (0, ... N - 1) and j from (i+1, ... N). However, we should consider that the a *force* added to i, should cause a counter force added to j. 

##### Removing pow
Raising powers is expensive for computers! While very architecture dependent here is an overview of clock ticks per operation on floating points:
- Addition / subtraction: 3 cycles
- Multiplication: 3 cycles
- Division: 10-20 cycles
- Square root: 10-20 cycles
- pow, exp, sin, cos, log: 20-100+ cycles

This is why it can be beneficial to prevent the amount of uses of the pow function. In this case we can easily get rid of the std::sqrt function, by working with $r^2$. Furthermore, we can get rid std::pow(r, -12) function by computing $r^{-2}$ and then $r^{-6}$. Precomputing $r^{-6}$ costs one division and 3 multiplications.

#### Benchmarking
Don't forget to rebuild with `pip install .` in the main directory and restarting the notebook kernel. You can use jupyters timing function `%%timeit` to benchmark your implementations. This function will run the code 7 times and report the average time it took to complete. 

In [ ]:
%%timeit
md = MolecularDynamics(
    numberOfParticles=800,
    temperature=1.0,
    dt=0.005,
    boxSize=np.cbrt(16.0) * 8.0,
    logLevel=0,
    seed=12,
    numberOfEquilibrationSteps=2000,
    numberOfProductionSteps=2000,
    sampleFrequency=100,
)
md.run()

In [ ]:
npart = [100, 200, 400, 800, 1600]
times = [0.0924, 0.392, 1.28, 3.94, 12.2]
fig, ax = plt.subplots()
# ax.set_xscale("log")
# ax.set_yscale("log")
ax.scatter(npart, times)